# Scaling computations using parallel computing

## Przemysław Szufel

<a class="anchor" id="toc"></a>
## Table of content
  

1. [Multithreading](#multithreading)
2. [Green threading](#green)
3. [Multi-processing and distributed computing](#multiprocessing)
4. [(background material) Parallelize via Single Instruction Multiple Data (SIMD)](#simd)

Before running Jupyter notebook set in Julia number of threads.
This should be done *before* actually running the `notebook()` command.
The number of threads can be also set up in Julia options in Visual Studio code (if this is used to run this notebook).
```
# run this code from Julia console just before starting Jupyter Notebook
ENV["JULIA_NUM_THREADS"]=4
```

In [ ]:
println("Number of threads that your Julia is run: ## $(Threads.nthreads())")

In [ ]:
using BenchmarkTools, Distributed

<a class="anchor" id="multithreading"></a>
### Multithreading
---- [Return to table of contents](#toc) ---


In [ ]:
Threads.nthreads()

In [ ]:
function ssum(x)
    r, c = size(x)
    y = zeros(c)
    for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end

In [ ]:
function tsum(x)
    r, c = size(x)
    y = zeros(c)
    Threads.@threads for i in 1:c
        for j in 1:r
            @inbounds y[i] += x[j, i]
        end
    end
    y
end


In [ ]:
x = rand(1000,10000);

In [ ]:
@time ssum(x)
@time ssum(x);

In [ ]:
@time tsum(x)
@time tsum(x);

#### Locking mechanism for threads

In [ ]:
function f_bad()
    x = 0
    Threads.@threads for i in 1:10^6
        x += 1
    end
    return x
end


In [ ]:
f_bad()

In [ ]:
function f_add()
    x = 0 
    for i in 1:10^6
        x += 1
    end
    x
end
@btime f_add()
    

In [ ]:
function f_atomic()
    x = Threads.Atomic{Int}(0)
    Threads.@threads for i in 1:10^6
        Threads.atomic_add!(x, 1)
    end
    return x[]
end
f_atomic()

In [ ]:
function f_spin()
    l = Threads.SpinLock()
    x = 0
    Threads.@threads for i in 1:10^6
        Threads.lock(l) do
            x += 1
        end
    end
    return x
end

function f_reentrant()
    l = ReentrantLock()
    x = 0
    Threads.@threads for i in 1:10^6
        Threads.lock(l) do
            x += 1
        end
    end
    return x
end


In [ ]:
using DataFrames
stats = DataFrame()
for f in [f_bad, f_atomic, f_spin, f_reentrant]
    for i = 1:2
        value, elapsedtime  = @timed f()
        push!(stats, (f=string(f),i=i, value=value, timems=elapsedtime*1000))
    end
end
println(stats)


<a class="anchor" id="green"></a>
### Green threading 
---- [Return to table of contents](#toc) ---


In [ ]:
@time sleep(2)

In [ ]:
@time t = @async sleep(4)

In [ ]:
t

In [ ]:
function dojob(i)
    val = round(rand(), digits=2)
    sleep(val)   # this could be external computations with I/O
    i, val
end

In [ ]:
result = Vector{Tuple{Int,Float64}}(undef, 8)

In [ ]:
@time for i=1:8
    result[i] = dojob(i)
end
result

In [ ]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time for i=1:8
   @async result[i] = dojob(i)
end
result

In [ ]:
result = Vector{Tuple{Int,Float64}}(undef, 8);
@time @sync for i=1:8
   @async result[i] = dojob(i)
end
result

<a class="anchor" id="multiprocessing"></a>
### Multi-processing and distributed computing
---- [Return to table of contents](#toc) ---


In [ ]:
using Distributed

This code adds 4 workers (and avoids adding more)

In [ ]:
addprocs(max(0, 5-nprocs()));

In [ ]:
workers()

In [ ]:
function s_rand()
    n = 10^4
    x = 0.0
    for i in 1:n
        x += sum(rand(10^4))
    end
    x / n
end
 
@time s_rand()
@time s_rand()


In [ ]:
using Distributed
 
function p_rand()
    n = 10^4
    x = @distributed (+) for i in 1:n
        # the last line will be aggregated
        sum(rand(10^4))
    end
    x / n
end

@time p_rand()
@time p_rand()


In [ ]:
workers()'

In [ ]:
fetch(@spawnat 3 4+3)

In [ ]:
function myf() 
    println("I am on worker ", myid())
    rand()
end
myf()

In [ ]:
a = nothing
try 
    fetch(@spawnat 4 myf())
catch e
    println(e)
end

In [ ]:
@everywhere function myf() 
    println("I am on worker ", myid())
    rand()
end
fetch(@spawnat 4 myf())

#### A typical pattern for setting an intial state across workers

In [ ]:
using Distributed
@everywhere using Pkg
@everywhere Pkg.activate(".")
@everywhere using Distributed, Random, DataFrames

@everywhere function calc(x, y)
    2x + y
end

@everywhere function init_worker()    
   Random.seed!(myid())
    # reading initial data from files or other actions
end

@sync for wid in workers()
    @async fetch(@spawnat wid init_worker())
end


Typically results are collected to a `DataFrame`

In [ ]:
data = @distributed (append!) for (i, j) = vec(collect(Iterators.product(1:4, 1:3)))
    a = rand(1:499)
    b = rand(1:9)*1000
    c = calc(a, b)
    DataFrame(;i,j,a,b,c,procid = myid())
end

<a class="anchor" id="simd"></a>
### (background material) Parallelize via Single Instruction Multiple Data (SIMD)
---- [Return to table of contents](#toc) ---



In [ ]:
function dot1(x, y)
    s = 0.0
    for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

In [ ]:
function dot2(x, y)
    s = 0.0
    @simd for i in 1:length(x)
        @inbounds s += x[i]*y[i]
    end
    s
end

In [ ]:
x = 100*rand(10000)
y = 100*rand(10000);

@btime dot1($x, $y)
@btime dot2($x, $y)

In [ ]:
res1 =  dot1(x, y)

In [ ]:
res2 =  dot2(x, y)

In [ ]:
res1 == res2

In [ ]:
@show res1 
@show res2